In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = './.google/credentials/google_credentials.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/05/19 14:23:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df = spark.read.parquet('gs://data_lake_minhnghia113/tiki_book_data.parquet')

In [6]:
df.count()

1975

In [7]:
df.registerTempTable('trips_data')

/home/minhnghia/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    publisher_vn,

    -- Revenue calculation 
    COUNT(DISTINCT name) AS book_publisher,
    SUM(COALESCE(price,0) * COALESCE(quantity,0)) AS magin_profit,
    COUNT(DISTINCT manufacturer) AS manufacturer_publissher
FROM
    trips_data
GROUP BY
    1
ORDER BY COUNT(DISTINCT name) DESC
""")

In [18]:
from pyspark.sql import functions as F

In [23]:
df_result.coalesce(1).write.parquet('gs://data_lake_minhnghia113/report/', mode='overwrite')

In [ ]:
df_result.coalesce(1).write.format('bigquery').option('table','aesthetic-root-416403.tiki.report').